In [2]:
import os
import shutil
import json
from tqdm import tqdm

# Paths
coco_path = r'C:\\Users\\xxnab\\fiftyone\\coco-2017\\train'
output_path = r'C:\\Users\\xxnab\\fiftyone\\coco-2017\\tensorflow_dataset'

# Load annotations
with open(os.path.join(coco_path, 'labels.json')) as f:
    annotations = json.load(f)

# Create class directories
categories = {cat['id']: cat['name'] for cat in annotations['categories']}
for cat in categories.values():
    os.makedirs(os.path.join(output_path, 'train', cat), exist_ok=True)

# Move images to class directories
for ann in tqdm(annotations['annotations']):
    img_id = ann['image_id']
    cat_id = ann['category_id']
    img_name = f'{img_id:012d}.jpg'
    src_path = os.path.join(coco_path, 'data', img_name)
    dst_path = os.path.join(output_path, 'train', categories[cat_id], img_name)
    shutil.copy(src_path, dst_path)

print("COCO-2017 dataset has been successfully converted to TensorFlow ImageFolder dataset.")


100%|██████████| 860001/860001 [25:18<00:00, 566.39it/s]  

COCO-2017 dataset has been successfully converted to TensorFlow ImageFolder dataset.


In [3]:
import os
import shutil
import json
import random
from tqdm import tqdm

# Paths
coco_path = r'C:\\Users\\xxnab\\fiftyone\\coco-2017\\train'
output_path = r'C:\\Users\\xxnab\\fiftyone\\coco-2017\\tensorflow_dataset'

# Load annotations
with open(os.path.join(coco_path, 'labels.json')) as f:
    annotations = json.load(f)

# Create class directories for train, validation, and test
categories = {cat['id']: cat['name'] for cat in annotations['categories']}
for split in ['train', 'validation', 'test']:
    for cat in categories.values():
        os.makedirs(os.path.join(output_path, split, cat), exist_ok=True)

# Split ratios
train_ratio = 0.7
val_ratio = 0.15
test_ratio = 0.15

# Shuffle and split the annotations
random.shuffle(annotations['annotations'])
total_annotations = len(annotations['annotations'])
train_end = int(train_ratio * total_annotations)
val_end = int((train_ratio + val_ratio) * total_annotations)

train_annotations = annotations['annotations'][:train_end]
val_annotations = annotations['annotations'][train_end:val_end]
test_annotations = annotations['annotations'][val_end:]

# Function to move images to the respective directories
def move_images(annotations, split):
    for ann in tqdm(annotations, desc=f'Moving {split} images'):
        img_id = ann['image_id']
        cat_id = ann['category_id']
        img_name = f'{img_id:012d}.jpg'
        src_path = os.path.join(coco_path, 'data', img_name)
        dst_path = os.path.join(output_path, split, categories[cat_id], img_name)
        shutil.copy(src_path, dst_path)

# Move images to the respective directories
move_images(train_annotations, 'train')
move_images(val_annotations, 'validation')
move_images(test_annotations, 'test')

print("COCO-2017 dataset has been successfully split into train, validation, and test sets.")


Moving test images: 100%|██████████| 129001/129001 [06:06<00:00, 351.79it/s]

COCO-2017 dataset has been successfully split into train, validation, and test sets.


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

# Paths
train_dir = r'C:\\Users\\xxnab\\fiftyone\\coco-2017\\tensorflow_dataset\\train'
val_dir = r'C:\\Users\\xxnab\\fiftyone\\coco-2017\\tensorflow_dataset\\validation'
test_dir = r'C:\\Users\\xxnab\\fiftyone\\coco-2017\\tensorflow_dataset\\test'

# Image data generators
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

# Load the MobileNetV2 model
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add custom layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x)

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator
)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_generator)
print(f'Test Loss: {test_loss}')
print(f'Test Accuracy: {test_accuracy}')

# Get true labels and predictions
test_generator.reset()
Y_pred = model.predict(test_generator)
y_pred = np.argmax(Y_pred, axis=1)
y_true = test_generator.classes

# Classification report
print('Classification Report')
print(classification_report(y_true, y_pred, target_names=test_generator.class_indices.keys()))

# Confusion matrix
print('Confusion Matrix')
print(confusion_matrix(y_true, y_pred))


In [6]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

# Paths
train_dir = r'C:\\Users\\xxnab\\fiftyone\\coco-2017\\tensorflow_dataset\\train'
val_dir = r'C:\\Users\\xxnab\\fiftyone\\coco-2017\\tensorflow_dataset\\validation'
test_dir = r'C:\\Users\\xxnab\\fiftyone\\coco-2017\\tensorflow_dataset\\test'

# Image data generators
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

# Load the MobileNetV2 model
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add custom layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x)

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_accuracy', patience=3, restore_best_weights=True)

# Train the model with early stopping
model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator,
    callbacks=[early_stopping]
)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_generator)
print(f'Test Loss: {test_loss}')
print(f'Test Accuracy: {test_accuracy}')

# Get true labels and predictions
test_generator.reset()
Y_pred = model.predict(test_generator)
y_pred = np.argmax(Y_pred, axis=1)
y_true = test_generator.classes

# Classification report
print('Classification Report')
print(classification_report(y_true, y_pred, target_names=test_generator.class_indices.keys()))

# Confusion matrix
print('Confusion Matrix')
print(confusion_matrix(y_true, y_pred))


Found 342996 images belonging to 80 classes.
Found 95879 images belonging to 80 classes.
Found 95894 images belonging to 80 classes.
Epoch 1/10
10719/10719 ━━━━━━━━━━━━━━━━━━━━ 4586s 427ms/step - accuracy: 0.2625 - loss: 2.6706 - val_accuracy: 0.3151 - val_loss: 2.3428
Epoch 2/10
10719/10719 ━━━━━━━━━━━━━━━━━━━━ 4029s 376ms/step - accuracy: 0.2814 - loss: 2.4755 - val_accuracy: 0.3331 - val_loss: 2.2944
Epoch 3/10
10719/10719 ━━━━━━━━━━━━━━━━━━━━ 3946s 368ms/step - accuracy: 0.2854 - loss: 2.4425 - val_accuracy: 0.3210 - val_loss: 2.2690
Epoch 4/10
10719/10719 ━━━━━━━━━━━━━━━━━━━━ 4532s 423ms/step - accuracy: 0.2863 - loss: 2.4228 - val_accuracy: 0.3270 - val_loss: 2.2444
Epoch 5/10
10719/10719 ━━━━━━━━━━━━━━━━━━━━ 4054s 378ms/step - accuracy: 0.2861 - loss: 2.4039 - val_accuracy: 0.3289 - val_loss: 2.2410
2997/2997 ━━━━━━━━━━━━━━━━━━━━ 1246s 416ms/step - accuracy: 0.3328 - loss: 2.2903
Test Loss: 2.2829558849334717
Test Accuracy: 0.3355267345905304
2997/2997 ━━━━━━━━━━━━━━━━━━━━ 887s 

c:\Users\xxnab\OneDrive\Documents\GitHub\FYP\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\xxnab\OneDrive\Documents\GitHub\FYP\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\xxnab\OneDrive\Documents\GitHub\FYP\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{m